In [3]:
!pip install regionmask geopandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 12.4 MB/s eta 0:00:00


In [4]:
#Importação das bibliotecas
import gcsfs 
import intake
import xarray as xr
import pandas as pd
import cftime
import geopandas as gpd
import regionmask
import re
import numpy as np
import fiona
fiona.drvsupport.supported_drivers['KML'] = 'rw'

In [45]:
def rename_coords(ds):
    """Renomeia as variáveis de latitude, longitude e profundidade para 'lat', 'lon' e 'lev',
    respectivamente, usando os nomes de variáveis de coordenadas encontrados automaticamente no arquivo.
    """
    # Cria um dicionário com os possíveis nomes antigos das variáveis de latitude, longitude e profundidade
    # e seus respectivos novos nomes
    coord_names = {
        'latitude': 'lat', 'nav_lat': 'lat', 'lat': 'lat',
        'longitude': 'lon', 'nav_lon': 'lon', 'lon': 'lon',
        'olevel': 'lev',
        'olevel_bounds': 'lev_bnds'
    }
    # Itera sobre a lista de nomes de coordenadas presentes no arquivo
    for coord_name in ds.coords.keys():
        # Verifica se o nome da coordenada corresponde a um dos possíveis nomes antigos das variáveis de coordenadas
        if coord_name in coord_names:
            # Renomeia a variável de coordenada usando o método rename()
            ds = ds.rename({coord_name: coord_names[coord_name]})
    # Retorna o Dataset com as variáveis de coordenadas renomeadas
    return ds.copy()

#Função para acertar a variável tempo! Pois alguns formatos disponiveis no CMIP6 para a variável time dificultam sua manipulação. 
def to_360day_monthly(da):
    ''' Conversão da dimensão de tempo de modelos climáticos.
        Função criada por Claire Carouge no CLEX CMS Blog'''
    val = da.copy()
    time1 = da.time.copy()
    for itime in range(val.sizes['time']):
        bb = val.time.values[itime].timetuple()
        time1.values[itime] = cftime.Datetime360Day(bb[0],bb[1],16)
    val = val.assign_coords({'time':time1})
    return val

#Função que insere a varíavel área no meu xarray para cálculo de volume
def assing_area(dic):
    pesquisa = {'source_id': dic['source_id'],
                'table_id': "Ofx",
                'variable_id': 'areacello',
                'experiment_id': dic['experiment_id'],
                'member_id': dic['member_id']}
    
    cat_area = cmip6.search(require_all_on='source_id', **pesquisa)
    cat_area = cat_area.to_dataset_dict(aggregate=True,
                            storage_options={'token': 'anon'},
                            xarray_open_kwargs={'consolidated': True,
                            'decode_times': True,
                            'use_cftime': True})
    
    #Caso não retorne nada retorna None
    if len(cat_area)==0:
        return None
    
    #Seleciona a áera do grid gn
    for key in list(cat_area.keys()):
        if ".gn" in key:
            ds_area = cat_area[key]
        
    ds_area = ds_area.squeeze().drop(["member_id", "dcpp_init_year"])
    ds_select_area = ds_area["areacello"]
    return rename_coords(ds_select_area)

#Função que converte a profundidade de centímetros para metro.
def depth_m(ds):
    if "lev" in ds:
        if "units" in ds["lev"].attrs:
            units = ds["lev"].units.lower()
            if units == "cm" or units == "centimeters":
                ds["lev"] = ds["lev"] / 100
                ds["lev"].attrs["units"] = "m"
    return ds

#Renomeia a variável level_bnds
def rename_lev_bnds(ds):
    if 'axis_nbounds' in ds.dims:
        ds = ds.rename_dims({'axis_nbounds': 'bnds'})
    elif 'd2' in ds.dims:
        ds = ds.rename_dims({'d2': 'bnds'})

    return ds

#Função para calcular a média de determinada variavel
def media(ds, lev_1, lev_2): 
    ds = ds.sel(lev=slice(lev_1, lev_2))

    # Crie uma máscara booleana para identificar onde a temperatura é não nula
    temperature_not_null = ds['thetao'].notnull()

    # Aplique a máscara à variável de volume para obter um volume ponderado válido
    weighted_volume = ds['vol'] * temperature_not_null

    # Calcule as médias ponderadas para temperatura e salinidade em relação ao volume válido
    media_thetao = ((ds['thetao'] * ds['vol']).sum() /  weighted_volume.sum()).compute()

    # Crie uma máscara booleana para identificar onde a temperatura é não nula
    so_not_null = ds['so'].notnull()

    # Aplique a máscara à variável de volume para obter um volume ponderado válido
    weighted_volume = ds['vol'] * so_not_null

    # Calcule as médias ponderadas para temperatura e salinidade em relação ao volume válido
    media_so = ((ds['so'] * ds['vol']).sum() /  weighted_volume.sum()).compute()
    
    return media_thetao, media_so

In [6]:
#Acessa o conjunto de metadados do CMIP6 hospedados no Google Cloud.
cmip6 = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

In [7]:
#Lendo a minha Tabela de Modelo.
#Verifique o caminho, no meu jupyter lab está tudo na mesma pasta.
df = pd.read_excel("Tabela_Modelos/Tabela_Modelos.xlsx", sheet_name = 1)
df = df.rename(columns = lambda x : x.strip())

In [8]:
#Realiza minha pesquisa de acordo com a minha Tabela de Modelos
pesquisas = []
for index, row in df.iterrows():
    pesquisa = {'source_id': row['source_id'],
                'table_id': row['table_id'],
                'variable_id': row['variable_id'].split(', '),
                'experiment_id': row['experiment_id'],
                'member_id': row['member_id']}
    pesquisas.append(pesquisa)

In [9]:
gdf = gpd.read_file('Area_Projeto/50S_20S/50S_20S.kml', driver="KML")

In [10]:
import os
import pandas as pd

# Define o slice de tempo inicial
time_inicio = 1970
time_fim = 1980

# Realiza um for para selecionar minha área de interesse para todos os Modelos
for pesquisa in pesquisas:
    cat = cmip6.search(require_all_on='source_id', **pesquisa)
    cat = cat.to_dataset_dict(aggregate=True,
                            storage_options={'token': 'anon'},
                            xarray_open_kwargs={'consolidated': True,
                            'decode_times': True,
                            'use_cftime': True})
    
    for key in list(cat.keys()):
        if ".gn" in key:
            ds = cat[key]
            
            #Area do modelo.
            ds_area = assing_area(pesquisa)

            if ds_area is None:
                continue

            #Corrigindo os times. Utiliza a função definida anteriormente para correção da variável time
            ds = to_360day_monthly(ds)

            #Renomeia as variaves para uniformizar o nome nos modelos
            ds = rename_coords(ds)

            #Renomeia o lev_bnds
            ds = rename_lev_bnds(ds)

            #Trasnforma as unidades da profundidade para metros
            ds = depth_m(ds)

            #Deleta as variáveis presentes no xarray que não são do nosso interesse.
            ds_drop = ds.drop([v for v in ds.coords if v not in ['lat', 'lon', 'time', 'lev', 'lev_bnds']])

            #Adiciona a variável área ao meu xarray
            ds_with_area = ds_drop.assign_coords(area=ds_area)

            #Define um slice do time
            ds_time = ds_with_area.sel(time=slice(str(time_inicio), str(time_fim)))

            #Faz o squeeze para tirar dimensões não importantes
            ds_time = ds_time.squeeze()

            #Converte o GeoDataFrame para um objeto region mask
            mask = regionmask.mask_geopandas(gdf, ds_time['lon'], ds_time['lat'])

            #Aplica a máscara ao dataset
            ds_masked = ds_time.where(mask==mask, drop=True)
            
            #Calcula a espessura
            espessura = ds_masked.lev_bnds.diff(dim='bnds').squeeze()
    
            #Coloca a variável espessura dentro do xarray
            ds_lev = ds_masked.assign(espessura=espessura)

            #Calcula o volume 
            ds_lev["vol"] = ds_lev["espessura"] * ds_lev["area"]
            
            results = []
            
            for i in ds_lev.time:
                
                ds_t = ds_lev.sel(time=i)
                
                mean_t, mean_s = media(ds_t, 500, 2000)

                results.append({
                    'time': i.values,
                    'media_thetao': mean_t.values,
                    'media_so': mean_s.values
                })

            # Crie um DataFrame a partir dos resultados
            result_df = pd.DataFrame(results)
            
            nome = key.split('.')
            
            chave = '.'.join(nome[0:4])
            
            result_df.to_csv("Medias_Temporais/50S_20S/500_2000m/{}.csv".format(chave))
            
            print("Processamento Concluído para o dataframe {}".format(key))


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


/srv/conda/envs/notebook/lib/python3.10/site-packages/intake_esm/_search.py:80: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for _, group in grouped:


KeyboardInterrupt: 